# Practica 1 APC: Graduate Admission 2
## Predicting admission from important parameters

- #### Bruno Moya Ruiz - 1568176
- #### Marc Garrofé Urrutia - 
- #### Martí Simon Rojas - 

# Apartat (C): Analitzant Dades
En aquest apartat analitzarem la nostra base de dades.

In [6]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats

# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('data/Admission_Predict_Ver1.1.csv')
data = dataset.values

x = data[:, :2]
y = data[:, 2]

print("Dimensionalitat de la BBDD:", dataset.shape)
print("Dimensionalitat de les entrades X", x.shape)
print("Dimensionalitat de l'atribut Y", y.shape)

Dimensionalitat de la BBDD: (500, 9)
Dimensionalitat de les entrades X (500, 2)
Dimensionalitat de l'atribut Y (500,)


Ara mirarem si presenta valors no existents, es a dir que el seu valor sigui NaN.

In [7]:
print("Per comptar el nombre de valors no existents:")
print(dataset.isnull().sum())

Per comptar el nombre de valors no existents:
Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


In [8]:
print("Per visualitzar les primeres 5 mostres de la BBDD:")
dataset.head() 

Per visualitzar les primeres 5 mostres de la BBDD:


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.500,4.500,9.650,1,0.920
1,2,324,107,4,4.000,4.500,8.870,1,0.760
2,3,316,104,3,3.000,3.500,8.000,1,0.720
3,4,322,110,3,3.500,2.500,8.670,1,0.800
4,5,314,103,2,2.000,3.000,8.210,0,0.650


Podem observar estadístiques de la nostra base de dades, com la desviació standard d'aquesta (std) , la mitjana de cada variable (mean) entre altres:

In [5]:
dataset.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000
mean,250.500,316.472,107.192,3.114,3.374,3.484,8.576,0.560,0.722
std,144.482,11.295,6.082,1.144,0.991,0.925,0.605,0.497,0.141
min,1.000,290.000,92.000,1.000,1.000,1.000,6.800,0.000,0.340
25%,125.750,308.000,103.000,2.000,2.500,3.000,8.127,0.000,0.630
50%,250.500,317.000,107.000,3.000,3.500,3.500,8.560,1.000,0.720
75%,375.250,325.000,112.000,4.000,4.000,4.000,9.040,1.000,0.820
max,500.000,340.000,120.000,5.000,5.000,5.000,9.920,1.000,0.970
